In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
trips19 = pd.read_csv('tripdata_2019-05.csv')
trips20 = pd.read_csv('tripdata_2020-05.csv')

/home/khodor/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# для того, чтобы не засорять оперативу непонятными данными, удалим их на время
del_columns = ['VendorID', 'RatecodeID', 'store_and_fwd_flag', 'extra', 'mta_tax', 'tip_amount', 'improvement_surcharge', 'congestion_surcharge']
trips19 = trips19.drop(columns=del_columns)
trips20 = trips20.drop(columns=del_columns)

In [5]:
max_passengers_count = trips20['passenger_count'].value_counts().idxmax()
print("Max passengers count =", max_passengers_count)
trips20.passenger_count = trips20.passenger_count.fillna(max_passengers_count)

Max passengers count =  1.0


In [6]:
trips19['tpep_pickup_datetime'] = pd.to_datetime(trips19.tpep_pickup_datetime)
trips19['tpep_dropoff_datetime'] = pd.to_datetime(trips19.tpep_dropoff_datetime)

In [7]:
trips20['tpep_pickup_datetime'] = pd.to_datetime(trips20.tpep_pickup_datetime)
trips20['tpep_dropoff_datetime'] = pd.to_datetime(trips20.tpep_dropoff_datetime)

In [8]:
trips19 = trips19[trips19.tpep_pickup_datetime <= trips19.tpep_dropoff_datetime]
trips19 = trips19[trips19.PULocationID <= 263]
trips19 = trips19[trips19.DOLocationID <= 263]

In [9]:
trips20 = trips20[trips20.tpep_pickup_datetime <= trips20.tpep_dropoff_datetime]
trips20 = trips20[trips20.PULocationID <= 263]
trips20 = trips20[trips20.DOLocationID <= 263]

In [10]:
MILES_TO_KM = 1609.34

In [11]:
trips19['trip_pickup_day'] = trips19.tpep_pickup_datetime.dt.day
trips19['week_day'] = trips19.tpep_pickup_datetime.dt.dayofweek
trips19['trip_pickup_hour'] = trips19.tpep_pickup_datetime.dt.hour
trips19['trip_dropoff_hour'] = trips19.tpep_dropoff_datetime.dt.hour
trips19['trip_duration'] = trips19.tpep_dropoff_datetime - trips19.tpep_pickup_datetime
trips19['trip_duration_secs'] = trips19.trip_duration.dt.seconds
trips19['trip_distance'] = trips19['trip_distance'] * MILES_TO_KM
trips19['average_velocity'] = trips19['trip_distance'] / trips19['trip_duration_secs']

In [12]:
trips20['trip_pickup_day'] = trips20.tpep_pickup_datetime.dt.day
trips20['week_day'] = trips20.tpep_pickup_datetime.dt.dayofweek
trips20['trip_pickup_hour'] = trips20.tpep_pickup_datetime.dt.hour
trips20['trip_dropoff_hour'] = trips20.tpep_dropoff_datetime.dt.hour
trips20['trip_duration'] = trips20.tpep_dropoff_datetime - trips20.tpep_pickup_datetime
trips20['trip_duration_secs'] = trips20.trip_duration.dt.seconds
trips20['trip_distance'] = trips20['trip_distance'] * MILES_TO_KM
trips20['average_velocity'] = trips20['trip_distance'] / trips20['trip_duration_secs']

In [13]:
trips19 = trips19[trips19.average_velocity <= 50]
trips19 = trips19[trips19.average_velocity > 1]
trips19 = trips19.drop(trips19[(trips19.trip_distance == 0) & (trips19.total_amount > 0) & (trips19.trip_duration_secs > 0)].index)
trips19 = trips19[trips19.total_amount >= 0]

In [14]:
trips20 = trips20[trips20.average_velocity <= 50]
trips20 = trips20[trips20.average_velocity > 1]
trips20 = trips20.drop(trips20[(trips20.trip_distance == 0) & (trips20.total_amount > 0) & (trips20.trip_duration_secs > 0)].index)
trips20 = trips20[trips20.total_amount >= 0]

In [ ]:
trips19['is_holiday'] = [x.date().day == 27 for x in trips19['tpep_pickup_datetime']]
trips20['is_holiday'] = [x.date().day == 25 for x in trips20['tpep_pickup_datetime']]

In [15]:
#trips19 = trips19[trips19.week_day <= 4]
#trips20 = trips20[trips20.week_day <= 4]

### Отметим все выезды на карте.

Импортируем данные о точках в каждой зоне такси.
Заведём "укороченную" версию этой таблицы для мерджа.

In [16]:
taxi_zones = pd.read_csv('taxi_zones.csv')
taxi_zones.columns = taxi_zones.columns.str.strip()
taxi_zones = taxi_zones.drop(columns=['Shape_Leng', 'Shape_Area', 'zone', 'borough']).rename(columns={'X': 'lon', 'Y': 'lat'})

In [17]:
taxi_zones

,lon,lat,PULocationID,DOLocationID
0,-74.176786,40.689516,1,1
1,-73.826126,40.625724,2,2
2,-73.849479,40.865888,3,3
3,-73.977023,40.724152,4,4
4,-74.189930,40.550340,5,5
...,...,...,...,...
258,-73.856351,40.899103,259,259
259,-73.903713,40.746798,260,260
260,-74.012919,40.708976,261,261
261,-73.945830,40.776534,262,262


Добавим в таблицы с данными о поездках координаты посадки и высадки пассажиров. 

In [18]:
trips19 = pd.merge(trips19, taxi_zones, on='PULocationID', how='left')
trips19 = trips19.drop(columns=['DOLocationID_y'])
trips19 = trips19.rename(columns={'DOLocationID_x': 'DOLocationID'})

trips19 = pd.merge(trips19, taxi_zones, on='DOLocationID', how='left')
trips19 = trips19.drop(columns=['PULocationID_y'])
trips19 = trips19.rename(columns={'PULocationID_x': 'PULocationID'})

In [19]:
trips20 = pd.merge(trips20, taxi_zones, on='PULocationID', how='left')
trips20 = trips20.drop(columns=['DOLocationID_y'])
trips20 = trips20.rename(columns={'DOLocationID_x': 'DOLocationID'})

trips20 = pd.merge(trips20, taxi_zones, on='DOLocationID', how='left')
trips20 = trips20.drop(columns=['PULocationID_y'])
trips20 = trips20.rename(columns={'PULocationID_x': 'PULocationID'})

Переименуем для удобства колонки с долготой и широтой и удалим строки, в которых есть значения Nan.

In [20]:
trips19 = trips19.rename(columns={'lon_x': 'lon_pu', 'lat_x': 'lat_pu', 'lon_y': 'lon_do', 'lat_y': 'lat_do'})
trips19 = trips19.dropna(subset=trips19.columns)

trips20 = trips20.rename(columns={'lon_x': 'lon_pu', 'lat_x': 'lat_pu', 'lon_y': 'lon_do', 'lat_y': 'lat_do'})
trips20 = trips20.dropna(subset=trips20.columns)

Для визуализации создадим таблицу вида:

| Отправление/прибытие | id поездки | широта | долгота |
|----------------------|------------|--------|---------|

In [21]:
cols = ['lon_pu', 'lat_pu', 'lon_do', 'lat_do']

In [22]:
trips19_vis = trips19.filter(['lon_pu', 'lat_pu', 'lon_do', 'lat_do']).reset_index()
trips19_vis = trips19_vis.rename(columns={'index': 'id'})

tmp_trips = trips19_vis.loc[trips19_vis.index.repeat(len(cols) / 2), ['id']].reset_index(drop=True)
trips19_vis = tmp_trips.join(pd.DataFrame(trips19_vis[cols].to_numpy().reshape(-1,2), columns=['lon','lat']))

trips19_vis = trips19_vis.reset_index()
trips19_vis['label'] = np.where(trips19_vis['index'] % 2 == 0, 'Origin', 'Destination')
trips19_vis = trips19_vis[['label', 'id', 'lon', 'lat']]

trips19_vis.to_csv('trips19_vis.csv', index=False)

In [23]:
trips20_vis = trips20.filter(['lon_pu', 'lat_pu', 'lon_do', 'lat_do']).reset_index()
trips20_vis = trips20_vis.rename(columns={'index': 'id'})

tmp_trips = trips20_vis.loc[trips20_vis.index.repeat(len(cols) / 2), ['id']].reset_index(drop=True)
trips20_vis = tmp_trips.join(pd.DataFrame(trips20_vis[cols].to_numpy().reshape(-1,2), columns=['lon','lat']))

trips20_vis = trips20_vis.reset_index()
trips20_vis['label'] = np.where(trips20_vis['index'] % 2 == 0, 'Origin', 'Destination')
trips20_vis = trips20_vis[['label', 'id', 'lon', 'lat']]

trips20_vis.to_csv('trips20_vis.csv', index=False)

In [24]:
trips19

,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,payment_type,fare_amount,tolls_amount,total_amount,...,week_day,trip_pickup_hour,trip_dropoff_hour,trip_duration,trip_duration_secs,average_velocity,lon_pu,lat_pu,lon_do,lat_do
0,2019-05-01 00:35:54,2019-05-01 00:37:27,1,1.50,145,145,2,3.0,0.0,4.30,...,2,0,0,00:01:33,93,25.957097,-73.948699,40.746590,-73.948699,40.746590
1,2019-05-01 00:44:57,2019-05-01 00:50:11,1,0.70,161,161,2,5.0,0.0,8.80,...,2,0,0,00:05:14,314,3.587701,-73.977432,40.758226,-73.977432,40.758226
2,2019-05-01 00:59:48,2019-05-01 01:10:22,1,2.00,163,141,1,9.5,0.0,15.30,...,2,0,1,00:10:34,634,5.076782,-73.978367,40.764425,-73.959713,40.766839
3,2019-05-01 00:23:20,2019-05-01 00:32:57,1,2.50,260,56,2,10.0,0.0,11.30,...,2,0,0,00:09:37,577,6.972877,-73.903713,40.746798,-73.859053,40.741599
4,2019-05-01 00:23:20,2019-05-01 00:32:57,1,2.50,260,56,2,10.0,0.0,11.30,...,2,0,0,00:09:37,577,6.972877,-73.903713,40.746798,-73.853384,40.752316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5300526,2019-05-31 23:40:41,2019-05-31 23:45:16,1,0.44,163,142,2,4.0,0.0,7.80,...,4,23,23,00:04:35,275,2.574944,-73.978367,40.764425,-73.981352,40.773906
5300527,2019-05-31 23:47:35,2019-05-31 23:57:15,1,1.39,43,229,1,8.0,0.0,14.16,...,4,23,23,00:09:40,580,3.856867,-73.965572,40.782460,-73.965174,40.756589
5300528,2019-05-31 23:22:08,2019-05-31 23:58:35,3,15.69,231,153,1,46.5,0.0,62.88,...,4,23,23,00:36:27,2187,11.545745,-74.006712,40.718696,-73.911063,40.875600
5300529,2019-05-31 23:10:12,2019-05-31 23:17:56,1,1.42,262,237,1,7.5,0.0,14.12,...,4,23,23,00:07:44,464,4.925135,-73.945830,40.776534,-73.965691,40.768542


In [ ]:
trips20